<h1><font color='DarkBlue'>PRACTICAL PHYLOGENETICS NOTEBOOK</font></h1>
<hr>
Dr Dave Lunt d.h.lunt@hull.ac.uk

<h2><font color='Blue'>Goals of these experiments</font></h2>

This Jupyter notebook will take you through two case studies using phylogenetic analysis to understand biological questions. This will involve aligning sequences and building maximum likelihood phylogenetic trees, followed by annotation an interpretation.

We hope that this will give you 
- experience in analysing DNA sequence data
- Understanding of the steps involved in phylogenetics
- Knowledge about the compleities of the specific case studies we are using

You will write up one of the case study analyses you perform today for your assessment.

<h2><font color='Blue'>Introduction to Jupyter computational notebooks</font></h2>

<font color=red>**FIRSTLY, DO NOT PANIC. EVERYTHING YOU NEED TO KNOW ABOUT COMPUTERS AND CODE WILL BE TAUGHT HERE. YOU WILL BE ABLE TO DO THIS EVEN IF YOU HAVE LITTLE EXPERIENCE WITH COMPUTERS**</font>

The class has mixed experience however, so if you have not done the bioinformatics practicals with me in Genetic Analysis last semester then please make yourself known and we will give you a 5 minute catch-up to make life easier.

This document that you are reading now is a **Jupyter Notebook**. It is a web browser based text editor that is also able to execute scripts ie  code. Today we are using the programming language `python`, probably the most used language in bioinformatics, but we could also run `R`, `bash` or many other languages. Scripts are found in the grey cells (see below) and have `In [ ]:` to their left in the margin..  

To execute a script, click the cell below and then press SHIFT+ENTER, or instead the `>|Run` button in the tool bar above. Try running this code below now 

In [1]:
print('Hey there, good job in running the python print command!')

Hey there, good job in running the python print command!


Can you identify which parts of this notebook are code, which parts output, and which parts documentation like this sentence? Discuss with us if you are in doubt.

1. Try editing the code and re-running. Replace "Good job" with "Even better job"
2. Instead of the `Run` button at the top you can click in the cell and press Shift-Enter to run the code. Most people find this faster, edit the cell below then give it a try:

In [2]:
print('Hey there, good job!')

Hey there, good job!


<h4><font color='Blue'>ACTION:</font></h4>

Now edit the cell above to have two print statements. On a new line type `print('Your new phrase')` and then run it. It might be easier to copy/paste and just change the pasted phrase. If it doesn't run well, you have a typo. Yes, its always a typo.

**Congratulations, you have now run, copy/pasted and edited cells. Those are all the skills you will need today**

This iterative edit-and-run approach is how much of modern biological data is explored and analysed. This mix of code and explanation you are seeing in this Jupyter notebook is called "literate programming"

This notebook will take you through the anaysis of the two case studies found in the practical handbook. To make this notebook concise, background information is excluded from it and only available in the practical handbook, and you will need to work with both documents. For each case study you will need to run several cells just as you did above. The programs will then align and clean the DNA sequences, build a tree and annotate it. **In most cases you will only need to run the cell just as you did above. In a few cases you will be able to tweak the script just a bit following clear instructions**. Good luck!

<h1><font color='Blue'>A NEW SPECIES OF APE?</font></h1>

![orangutan males](images/Bornean,_Sumatran_&_Tapanuli_orangs.jpg)

_Figure 1:_ Male Bornean, Sumatran and Tapanuli orangutans, three suggested species [wikipedia](https://en.wikipedia.org/wiki/Orangutan). 

The aim of today is to investigate what phylogenetics can tell us about diffrent species of great ape. It is, of course, complex. You might like to think how you would conceptually go about trying to get information using a phylogenetic approach.

_Table for quick reference showing latin names and common names for species used in this practical_ As always, Googling is encouraged.

| Name             | Common name           | Name  | Common Name |
| ----------------|:----------------------| --------------|:---------- |
| Macaca macaca | Macaque (outgroup)|  Homo sapiens sapiens | Modern humans
| Hylobates lar      | Gibbon (outgroup)     |  Homo sapiens neanderthalis  | Neanderthals (extinct)|
| Gorilla gorilla | Western Gorilla      | Homo sapiens denisovan | Denisovans (extinct)
| Gorilla beringei | Eastern/mountain Gorilla | Pongo abelii | Sumatran orangutan     |
| Pan troglodytes | Chimp      |   Pongo pygmaeus | Bornean orangutan   |
| Pan paniscus | Bonobo      |     Pongo tapanuliensis | Tapanuli orangutan|
|

<h2><font color='Blue'>How much data do you have?</font></h2>
Your working directory has some DNA sequence files in fasta format. There are a number of ways to determine the number of sequences in a file, here is a quick one-liner. 

Edit the cell to replace `name.fas` with the correct file `ape.fas`. Shift-Enter to run the cell as usual

In [3]:
!echo "Number of sequences: "; grep -c ">" name.fas

Number of sequences: 
grep: name.fas: No such file or directory


It should have displayed the number of sequences in the `ape.fas` file

Below we will use a few python packages to allow more complex analyses. In the next example we are going to find the number and total length of sequences using a useful code package called BioPython [1].

Remember: The code below has explanations of what each section does (explanations begin with the # symbol) as some people are interested in seeing bioinformatics code in action. **But you do not have to know python or understand this code. Just run the cell as usual.**

In [4]:
# --------------------------------------------
# Python code to report on number of sequences 
# in a file by using BioPython
# --------------------------------------------

# import BioPython code so we can use it
from Bio.SeqIO.FastaIO import SimpleFastaParser

# set counts to zero before starting
count = 0
total_len = 0

# open the data file and give it a handle (nickname)
with open("ape.fas") as in_handle:
    
# for each title line add 1 to count of records, 
# and add length of sequence to a count called total_len
     for title, seq in SimpleFastaParser(in_handle):
         count += 1
         total_len += len(seq)
            
# print the results in a readable format
print("The file contained %i records with total sequence length %i nucleotides" % (count, total_len))

The file contained 15 records with total sequence length 10721 nucleotides


<h4><font color='Blue'>QUESTIONS:</font></h4>

- Can you see which part of the above code specifies the fasta file `ape.fas`?

- How could you run this on a different file in the data directory called `testseqs.fasta`? 

<h4><font color='Blue'>ACTIONS:</font></h4>

Try it, just change the name above and re-run the cell, or ask for help if you can't quite see it. If you've done it correctly (watch for typos) then the number and length of sequence reported will change.

<hr>
<h2><font color='Blue'>Aligning the sequences</font></h2>
In order to carry out a valid analysis you have to align the DNA sequences. If you're not quite sure why, look at the images below and discuss with a demonstrator. 

![Aligned DNA sequence](./images/aligned.png "A DNA sequence alignment")
_A DNA sequence alignmnet. Each character (column) can be directly compared across the different species_

![Un-aligned DNA sequence](./images/unaligned.png "An incomplete  DNA sequence alignment")
_A set of DNA sequences not completely aligned. Each character (column) cannot be directly compared across the different species as some are 'shifted' so even though they are very similar, they look enormously different when just comparing down each column (charcter)_

To align the sequences we will use a program called MAFFT [2]. What piece of information will we have to add to the code? Yes, the name of the input DNA sequence file to be aligned.

<h4><font color='Blue'>ACTIONS:</font></h4>

- Change the name of the file in the following code to be `ape.fas`
- run the cell

In [5]:
# ---------------------------
# Align sequences using MAFFT
# ---------------------------

!mafft --auto --quiet ape.fas > ape.afa

<hr>
<h2><font color='Blue'>QC the alignment</font></h2>
Trimal [3] quality controls the alignment, removing badly aligned regions and alignment artefacts.

In [6]:
# ------------------------------------------
# Quality control the alignment using trimal
# ------------------------------------------

!trimal -in ape.afa -out ape_trimmed.afa -gappyout -keepheader

<hr>
<h2><font color='Blue'>Tree reconstruction</font></h2>
This section will reconstruct a maximum likelihood phylogenetic tree using the sequence alignment you have produced. We will use the program FastTree [4].

In [7]:
# -------------------------
# Build tree using FastTree
# -------------------------

!FastTree -gtr -nt ape_trimmed.afa > ape.nwk

FastTree Version 2.1.10 Double precision (No SSE3)
Alignment: ape_trimmed.afa
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Generalized Time-Reversible, CAT approximation with 20 rate categories
Initial topology in 0.00 seconds
Refining topology: 16 rounds ME-NNIs, 2 rounds ME-SPRs, 8 rounds ML-NNIs
Total branch-length 0.477 after 0.04 sec
ML-NNI round 1: LogLk = -1805.940 NNIs 2 max delta 0.00 Time 0.07
GTR Frequencies: 0.3150 0.3149 0.1056 0.2645ep 4 of 12   
GTR rates(ac ag at cg ct gt) 2.3845 14.9825 1.2609 1.5635 13.8255 1.0000
Switched to using 20 rate categories (CAT approximation)1 of 20   
Rate categories were divided by 0.732 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
Use -gamma for approximate but comparable Gamma(20) log-likelihoods
ML-NNI round 2: LogLk = -1589.440 NNIs 0 max delta 0.

<hr>
<h2><font color='Blue'>Tree Annotation and Viewing</font></h2>

The tree alone (below) is in bracket notation format (called Newick) and not very meaningful to examine.

```
((A,B),(C,D));
```
Instead we are going to display it as a graphic, and then annotate it to be easier to interpret. To do this we are going to use a tree graphics program called ToyTree [5].

<h4><font color='Blue'>QUESTION:</font></h4>

What treefile (.nwk) has just been written by the build tree cell above?

<h4><font color='Blue'>ACTION:</font></h4>

Take your newick treefile name and enter it into the cell below to replace "tree.nwk"

In [1]:
# -----------------------------------
# Drawing the phylogeny using ToyTree
# -----------------------------------
# import the code so we can use it here
import toytree       # a tree plotting library
import toyplot       # a general plotting library
# import numpy as np   # a numerical library, give it the shorthand 'np'

# read the newick format tree file, give it the name 'newick'
newick = "ape.nwk" # change this to point at your .nwk treefile
tre = toytree.tree(newick, tree_format=1)

tre.draw();

<svg class="toyplot-canvas-Canvas" height="342.0px" id="tf1d9484869f04328a58a9e323143205c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 370.0 342.0" width="370.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Pan_troglodytes5 Pan_troglodytes4 Pan_troglodytes3 Pan_troglodytes1 Pan_troglodytes6 Pan_paniscus3 Pan_paniscus2 Pan_paniscus1 Macaca_fascicularis Pongo_pygmaeus_pygmaeus1 Pongo_pygmaeus_pygmaeus4 Pongo_pygmaeus_pygmaeus3 Pongo_pygmaeus_pygmaeus2 Pongo_abelii2 Pongo_abelii3 Pongo_abelii1 Pongo_abelii5 Pongo_abelii4 Pongo_abelii6

If you see a graphic image of a phylogenetic tree, congratulations! If not please ask for a little help, its probably a quick fix for a demonstrator.

<h4><font color='Blue'>NOW ROOT THE TREE</font></h4>

Your tree will probably look very odd because it isn't yet rooted correctly. Use the next cell to root it by entering "Macaca" (Macacque) instead of "outgroup"

In [3]:
# ----------------
# Root and re-draw
# ----------------
# root and draw the tree
rtre = tre.root(wildcard="Macaca") # specify the outgroup taxon
rtre.draw(height=600, tip_labels_align=True); # draw the tree

<svg class="toyplot-canvas-Canvas" height="600.0px" id="te8e3f2b014b344e7b8fb88849f94bb6d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 370.0 600.0" width="370.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Pongo_abelii5 Pongo_abelii4 Pongo_abelii6 Pongo_abelii3 Pongo_abelii1 Pongo_abelii2 Pongo_pygmaeus_pygmaeus1 Pongo_pygmaeus_pygmaeus4 Pongo_pygmaeus_pygmaeus3 Pongo_pygmaeus_pygmaeus2 Pan_troglodytes5 Pan_troglodytes4 Pan_troglodytes3 Pan_troglodytes1 Pan_troglodytes6 Pan_paniscus3 Pan_paniscus2 Pan_paniscus1 Macaca_fascicularis

You should now have a tree that reveals a lot about the relationships betwen these species. It will be easier to interpret though when you put it into a report if you annotate and colour it by taxon.

<h4><font color='Blue'>NOW ANNOTATE THE TREE:</font></h4>
Although you now have 'the answer' it is not so easy to study this tree. You will need to compare the divergences between the two species of orangutan and compare those to the divergences between the two species of chimpanzee. In this simple tree its not too hard, but in general phylogeneticists label and colour to maintain focus on the correct comaprisons. You are now going to use the script below to colour in the tips by their species identity. 

Run the cell and examine the tree

In [17]:
# -----------------------------
# Colouring the tree label text
# -----------------------------

# set list of colours depending on the taxon label text
# numbers like #5384a3 are color hex codes (google it for other options)
colorlist = ["blue" if "Pan_paniscus" in tip
             else "darkblue" if "Pan_troglodytes" in tip 
             else "red" if "Pongo_abelii" in tip 
             else "brown" if "Pongo_pygmaeus" in tip
             else "#5384a3" for tip in rtre.get_tip_labels()] # cyan

canvas = rtre.draw(
    width=600,  # set dimensions of the figure
    height=600,
    scalebar=True,  # scale bar of divergence levels
    tip_labels_align=True,
    tip_labels=True,
    tip_labels_colors=colorlist,
    node_labels=None,
    node_sizes=[0 if i else 8 for i in rtre.get_node_values(None, 1, 0)],
    node_markers="s", # use "o" for circles instead of squares
    node_colors=toytree.colors[0],
)

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t893a13ba91984b38ae41b102be45ce59" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 600.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Pongo_abelii5 Pongo_abelii4 Pongo_abelii6 Pongo_abelii3 Pongo_abelii1 Pongo_abelii2 Pongo_pygmaeus_pygmaeus1 Pongo_pygmaeus_pygmaeus4 Pongo_pygmaeus_pygmaeus3 Pongo_pygmaeus_pygmaeus2 Pan_troglodytes5 Pan_troglodytes4 Pan_troglodytes3 Pan_troglodytes1 Pan_troglodytes6 Pan_paniscus3 Pan_paniscus2 Pan_paniscus1 Macaca_fascicularis 0.00 0.06 0.11 0.17 0.23 0.29

You now have all the skills to edit this script and change colours. Pick some ones you like and rerun.

<h3><font color=red>IMPORTANT, SAVE YOUR FILE</font></h3>
Make sure that you save and take away a copy of you tree file image in a format suitable to insert into your final report. Run the cell below and then find the file in your working directory and save it somewhere accesible.

In [13]:
# --------------------------------
# Save the tree as a graphics file
# --------------------------------

# import code to draw graphics files
import toyplot.pdf
import toyplot.svg
import toyplot.html

# draw graphics files
toyplot.svg.render(canvas, "ape.svg")
toyplot.pdf.render(canvas, "ape.pdf")
toyplot.html.render(canvas, "ape.html")

<h4><font color='Blue'>SPECIES DIFFERENTIATION</font></h4>
Looking at the tree, it would seem the two *Pan* clades are as distant from each other as the two *Pongo* clades
 
In order to examine this numerically we have created a figure (box and whisker plot) to show all the pairwise distances within Pong and Pan genera. What can we learn from this?

![Boxplot](./data/pairwise_tree_distances.png)



<hr>
<h2><font color='Blue'>A big data analysis of great apes</font></h2>

One very useful aspect of using code to carry out analyses is that once you have written it, and it works, its very little effort to re-run it again on any number or any size of other data sets.

Here I have collected from GenBank whole mitochondrial genomes (about 16,000 nucleotides) from a lot of great apes including humas, neanderthals, and species of gorillas in addition to the species you have just analysed. The file is large but we can just run the same code again. If you want to find out how much data you have, you could insert a cell and paste in the code to quantify sequences (from the ape example) and run it for the big_ape dataset. This is optional. For efficiency reasons I've compressed the code below a little, but its the same as you have just run.

This big analysis gives you the opportunity to decide whether the similarity of divergence between groups that you have just observed is true more widely. When the class have produced their big trees we will all discuss what the divergence levels might mean. If you want to find out how much data you are analysing you can copy and paste the "how much data do you have?" cell from above, and run it here (on the new fasta file). But that is optional.

Run this cell to align, trim the alignment, and then build a tree. It might take a few minutes to complete. When there is a number not an asterisk in the left margin then it is complete.

In [ ]:
# Align
!mafft --auto --quiet big_ape.fas > big_ape.afa
print("\nThe sequence alignment has finished")
# Trim
!trimal -in big_ape.afa -out big_ape_trimmed.afa -gappyout -keepheader
print("The alignment trimming has finished")
# Tree build
print("The phylogenetic tree construction has started\n")
!FastTree -gtr -nt big_ape_trimmed.afa > big_ape.nwk
print("\nThe phylogenetic analysis has finished")

Expect this to take a couple of minutes. Remember if there is an asterisk in the top left "`In [*]:`" then it is still working, when it is a number it is finished. 

If this completed without errors then you can just run the cell below and see the output tree. You may want to adjust colours and re-run a few times. If you had an errors, see if you can spot what wen't wrong, but seek assistance if not.

In [39]:
# import the code so we can use it here
import toytree       # a tree plotting library
import toyplot       # a general plotting library

# --------------------
# Read in the big tree
# --------------------
# read the newick format tree file, give it the name 'newick'
bignewick = "big_ape.nwk" # change this to point at your .nwk treefile
btre = toytree.tree(bignewick, tree_format=1)

# ----------------
# Root the tree
# ----------------
# root and draw the tree
brtre = btre.root(wildcard="Hylobates") # specify the outgroup taxon

# -----------------------------
# Colouring the tree label text
# -----------------------------

# set list of colours depending on the taxon label text
# numbers like #5384a3 are color hex codes (google it for other options)
colorlist = ["black" if "Hylobates" in tip
             else "darkblue" if "Pan" in tip 
             else "red" if "Pongo" in tip 
             else "green" if "Homo" in tip 
             else "brown" if "Gorilla" in tip 
             else "#5384a3" for tip in brtre.get_tip_labels()] # cyan

canvas, axes = brtre.draw(
    width=800,  # set dimensions of the figure
    height=1600,
    scalebar=True,  # scale bar of divergence levels
    tip_labels_align=True,
    tip_labels=True,
    tip_labels_colors=colorlist,
    node_labels=None,
    node_sizes=[0 if i else 8 for i in brtre.get_node_values(None, 1, 0)],
    node_markers="o", # use "o" for circles instead of squares
    node_colors=toytree.colors[0],
)

# canvas, axes = brtre.draw(width=400, height=800);

<svg class="toyplot-canvas-Canvas" height="1600.0px" id="td5ef4ff34b704e32962312f666458e0f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 1600.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Pan_troglodytes_ellioti_KU308531.1 Pan_troglodytes_ellioti_KU308533.1 Pan_troglodytes_ellioti_KU308532.1 Pan_troglodytes_ellioti_KU308530.1 Pan_troglodytes_ellioti_KU308529.1 Pan_troglodytes_verus_A993_KU308545.1 Pan_troglodytes_verus_B005_KU308547.1 Pan_troglodytes_verus_A994_KU308546.1 Pan_troglodytes_verus_N016_KU308548.1 Pan_troglodytes_troglodytes_B022_KU308541.1 Pan_troglodytes_troglodytes_B024_KU308542.1 Pan_troglodytes_troglodytes_B021_KU308540.1 Pan_troglodytes_troglodytes_B025_KU308543.1 Pan_troglodytes_troglodytes_A990_KU308539.1 Pan_paniscus_PP05_GU189658.1 Pan_paniscus_PP30_GU189676.1 Pan_paniscus_PP23_GU189661.1 Pan_paniscus_PP69_GU189670.1 Homo_sapiens_neanderthalensis_FM865410.1 Homo_sapiens_neanderthalensis_AM948965.1 Homo_sapiens_sapiens_AJ842750.1 Homo_sapiens_sapiens_FN600416.1 Homo_sapiens_denisova_KT780370.1 Homo_sapiens_denisova_KX663333.1 Gorilla_gorilla_gorilla_KU353721.1 Gorilla_gorilla_gorilla_EU095336.1 Gorilla_gorilla_gorilla_KF914214.1 Gorilla_gorilla_gorilla_KU353722.1 Gorilla_gorilla_gorilla_NC_011120.1 Gorilla_beringei_MH177751.1 Gorilla_beringei_MH177752.1 Gorilla_beringei_MH177753.1 Gorilla_beringei_MH177754.1 Pongo_abelii_CR247 Pongo_abelii_CR011 Pongo_abelii_B017 Pongo_abelii_B020 Pongo_abelii_KB550 Pongo_abelii_B018 Pongo_abelii_A949 Pongo_abelii_CRNA Pongo_abelii_A952 Pongo_abelii_KB4661 Pongo_abelii_CR519 Pongo_abelii_KB5883 Pongo_pygmaeus_CR535 Pongo_pygmaeus_CR536 Pongo_pygmaeus_A940 Pongo_pygmaeus_A938 Pongo_tapanuliensis_B019 Pongo_tapanuliensis_CRBT Pongo_tapanuliensis_KB9258 Hylobates_lar_HQ622776.1 0.00 0.03 0.06 0.09 0.12 0.15 0.18 0.20

In [40]:
# --------------------------------
# Save the tree as a graphics file
# --------------------------------

toyplot.svg.render(canvas, "bigape.svg")
toyplot.pdf.render(canvas, "bigape.pdf")
toyplot.html.render(canvas, "bigape.html")

**Or maybe the cell below. Merge them somehow**

In [41]:
# read the tree file
newick = "big_ape.nwk"
tre = toytree.tree(newick, tree_format=1)

# specify the outgroup to be Macaque
rtre = tre.root(wildcard="Hylobates")

# change these options until you are happy with the design
colorlist = ["#d6557c" if "Pan" in tip # pink
             else "blue" if "Gorilla_b" in tip #
             else "#4169E1" if "Gorilla_g" in tip #royalblue
             else "#008000" if "Homo_sapiens_sapiens" in tip #green
             else "#32CD32" if "neanderthal" in tip #limegreen
             else "#006400" if "denisovan" in tip #darkgreen
             else "red" if "Pongo_abelii" in tip #darkgreen
             else "orange" if "Pongo_pygmaeus" in tip #darkgreen
             else "brown" if "Pongo_tapanuliensis" in tip #darkgreen
             else "#5384a3" for tip in rtre.get_tip_labels()] # cyan

# CHANGE BELOW TO USE CANVAS

# draw the tree using these colours and some other standard options
canvas, axes =rtre.draw(
    height=1200,
    scalebar=True,
    node_labels=None,
    node_sizes=[0 if i else 8 for i in rtre.get_node_values(None, 1, 0)],
    node_markers="s",
    node_colors=toytree.colors[0], # could this be =colorlist too?
    tip_labels_align=True,
    tip_labels_colors=colorlist
);

# The following code cell is needed to save it as a graphics file

<svg class="toyplot-canvas-Canvas" height="1200.0px" id="td2db19eddb794fa0a0261ad4e0a8c66f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 465.0 1200.0" width="465.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Pan_troglodytes_ellioti_KU308531.1 Pan_troglodytes_ellioti_KU308533.1 Pan_troglodytes_ellioti_KU308532.1 Pan_troglodytes_ellioti_KU308530.1 Pan_troglodytes_ellioti_KU308529.1 Pan_troglodytes_verus_A993_KU308545.1 Pan_troglodytes_verus_B005_KU308547.1 Pan_troglodytes_verus_A994_KU308546.1 Pan_troglodytes_verus_N016_KU308548.1 Pan_troglodytes_troglodytes_B022_KU308541.1 Pan_troglodytes_troglodytes_B024_KU308542.1 Pan_troglodytes_troglodytes_B021_KU308540.1 Pan_troglodytes_troglodytes_B025_KU308543.1 Pan_troglodytes_troglodytes_A990_KU308539.1 Pan_paniscus_PP05_GU189658.1 Pan_paniscus_PP30_GU189676.1 Pan_paniscus_PP23_GU189661.1 Pan_paniscus_PP69_GU189670.1 Homo_sapiens_neanderthalensis_FM865410.1 Homo_sapiens_neanderthalensis_AM948965.1 Homo_sapiens_sapiens_AJ842750.1 Homo_sapiens_sapiens_FN600416.1 Homo_sapiens_denisova_KT780370.1 Homo_sapiens_denisova_KX663333.1 Gorilla_gorilla_gorilla_KU353721.1 Gorilla_gorilla_gorilla_EU095336.1 Gorilla_gorilla_gorilla_KF914214.1 Gorilla_gorilla_gorilla_KU353722.1 Gorilla_gorilla_gorilla_NC_011120.1 Gorilla_beringei_MH177751.1 Gorilla_beringei_MH177752.1 Gorilla_beringei_MH177753.1 Gorilla_beringei_MH177754.1 Pongo_abelii_CR247 Pongo_abelii_CR011 Pongo_abelii_B017 Pongo_abelii_B020 Pongo_abelii_KB550 Pongo_abelii_B018 Pongo_abelii_A949 Pongo_abelii_CRNA Pongo_abelii_A952 Pongo_abelii_KB4661 Pongo_abelii_CR519 Pongo_abelii_KB5883 Pongo_pygmaeus_CR535 Pongo_pygmaeus_CR536 Pongo_pygmaeus_A940 Pongo_pygmaeus_A938 Pongo_tapanuliensis_B019 Pongo_tapanuliensis_CRBT Pongo_tapanuliensis_KB9258 Hylobates_lar_HQ622776.1 0.00 0.07 0.14 0.20

Run this following cell to save your tree to a graphics format. You will need this for your report.

In [42]:
# change the output file names below and run the cell.
# remember to save these files and take them away
toyplot.svg.render(canvas, "big_ape2.svg")
toyplot.pdf.render(canvas, "big_ape2.pdf")

<h2><font color='Blue'>Well Done</font></h2>

You are now finished with case study 1, the apes. Case study two, the origins of HIV, will be much faster now you have experience.

Please feel free to take a short break here.
<hr>

<hr>
<h1><font color='Purple'>Case study 2</font></h1>
<h2><font color='DodgerBlue'>WHAT ARE THE ORIGINS OF HIV?</font></h2>
<h3><font color='DodgerBlue'>Sequence alignment and tree reconstruction</font></h3>

To determine whether HIV has coevolved with humans or if it has a recent zoonotic origin, the next script will reconstruct a tree from HIV and SIV sequences and will produce three figures.

You will create 3 output files, each of the same tree annotated with different metadata.
 

In [ ]:
#HIV

# Align
!mafft --auto --quiet SIVHIV_ENV.fas > SIVHIV_ENV.afa
print("\nThe sequence alignment has finished")
# Trim
!trimal -in SIVHIV_ENV.afa -out SIVHIV_ENV_trimmed.afa -gappyout -keepheader
print("The alignment trimming has finished")
# Tree build
print("The phylogenetic tree construction has started\n")
!FastTree -gtr -nt SIVHIV_ENV_trimmed.afa > SIVHIV_ENV.nwk
print("\nThe phylogenetic analysis has finished")

In [65]:
newick = "SIVHIV_ENV.nwk"
tre = toytree.tree(newick, tree_format=1)
rtre = tre.root(wildcard="ANT70")
canvas, axes = rtre.draw(
    width=600,  # set dimensions of the figure
    height=600,
    layout='c',
    edge_type='c',
    node_labels=None,
    node_sizes=[0 if i else 8 for i in rtre.get_node_values(None, 1, 0)],
    node_markers="s",
    node_colors=toytree.colors[0],
    tip_labels_align=True,
);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="tea07535cedc74a7b9c9a22796b503caa" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 600.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> DP943 MB897 MB66 U455 HXB2 LB7 CAM13 GAB1 TAN3 TAN2 TAN1 ANT YBF30 YBF106 EK505 US CAM5 CAM3 CP684 CP2135 P-RBF168 MVP5180 ANT70

**Insert questions and things to do before going**

<hr>
<h2><font color='Blue'>What skills have you acquired?</font></h2>

If you have completed this practical I think you have now showed your competency in a range of important practical and conceptual skills:
1. Understanding the use of phylogenetic trees
2. Basic use of Jupyter notebooks
3. Basic use of BioPython to characterise sequence data files
4. Basic use of python to align DNA sequecne data and build a phylogenetic tree
5. Use of python to programmatically annotate a phylogenetic tree

These are the sorts of phrase you could include on you cv if you wished.

## Software References

1. Cock PJA, Antao T, Chang JT, Chapman BA, Cox CJ, Dalke A, et al. Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics. 2009;25: 1422–1423. doi:10.1093/bioinformatics/btp163
2. Katoh K, Toh H. Recent developments in the MAFFT multiple sequence alignment program. Brief Bioinform. 2008;9: 286–298. doi:10.1093/bib/bbn013
3. Capella-Gutiérrez S, Silla-Martínez JM, Gabaldón T. trimAl: a tool for automated alignment trimming in large-scale phylogenetic analyses. Bioinformatics. 2009;25: 1972–1973. doi:10.1093/bioinformatics/btp348
4. Price MN, Dehal PS, Arkin AP. FastTree 2 – Approximately Maximum-Likelihood Trees for Large Alignments. PLoS ONE. 2010. p. e9490. doi:10.1371/journal.pone.0009490
5. Eaton DAR. Toytree: A minimalist tree visualization and manipulation library for Python. Methods Ecol Evol. 2020;11: 187–191. doi:10.1111/2041-210X.13313
